在计算机视觉的实际应用中，分类是远远不够的，更多的场景需要能同时探测器多个物体，不仅包括这些物体的分类，而且包括这些物体的位置和尺寸。通常使用包围盒（Bounding Box）来表示这些物体的位置和尺寸。

经典的使用神经网络实现目标检测的模型有：

- R-CNN
- SPPNet
- Fast R-CNN
- Faster R-CNN
- YOLO

# 1. [R-CNN](https://arxiv.org/abs/1311.2524)

R-CNN 模型发展到现在已经成为一个系列，从最初的 R-CNN 模型发展到 Fast R-CNN 模型，之后在 2015 年再一次突破，提出了 Faster R-CNN模型。近几年来，基于深度学习的目标检测方法发展很快，相继提出了很多精妙的思想，R-CNN 模型系列功不可没。R-CNN 模型由于提出较早，想必今天很多前沿方法，它的精度和效率都已经显现不足，但是该模型是很多算法的基础思想。

R-CNN 结合了 Selective Search 算法的区域建议框和由卷积神经网络产生的视觉丰富特征，它的性能比当时最好的检测模型在 Pascal VOC 2012 数据集上有将近 30% 的性能提升，mAP（mean average precision）指标从 40.9% 提升到了 53.3%。

R-CNN 模型的整体框架为：

- 采用 Selective Search 生成建议窗口（region proposals）
      每张图片大约生成 2000 个建议窗口，由于建议窗口尺寸不一，所以通过拉伸变换，形成统一的输入尺寸 227×227。

- 运用 CNN 网络进行特征提取
      将统一尺寸的建议窗图像输入 AlexNet 网络并提取 4096 维特征向量
      
- 对建议窗图像进行分类
      使用 SVM 分类器对 CNN 输出特征进行分类
      
- Bounding Box 回归
      用 Bounding Box 回归值校正原来的建议窗口，生成预测窗口坐标

![img](images/chapter15/R-CNN.png)

R-CNN 的训练步骤分为三步：

- 在 ImageNet 数据集上进行预训练（pre-training）

- 利用成熟的权重参数在 PASCAL VOC 数据集上进行 fine-tune

R-CNN 相比于目前的神经网络模型，精确度不够高，速度也较慢，无法满足实时系统需要。但 R-CNN 让业界第一次看到了深度学习在目标检测上的力量和潜力，是深度学习在目标检测任务上迈出的重要一步。

# 2. [SPPNet](https://arxiv.org/abs/1406.4729)

# 3. [Fast R-CNN](https://arxiv.org/abs/1504.08083)

Fast R-CNN 与 R-CNN 相比，训练时间从 84 小时减少为 9.5 小时，测试时间从 47 秒减少为 0.32 秒，在 PASCAL VOC 2007 数据集上的准确率约为 67%。